In [10]:
!pip freeze > requirements.txt

In [14]:
from konlpy.tag import Kkma    # 형태소 분석기 모듈(Hannanum, Kkma, Komoran, Okt)을 탑재
from tqdm.notebook import tqdm
import pandas as pd

#### change csv to parquet

In [15]:
for year in tqdm(range(1980, 2000)):
    df = pd.read_csv(f'./data/chosun_{year}.csv')

    df.to_parquet(f"./data/chosun_{year}.parquet",
                  engine="pyarrow", compression="gzip")

  0%|          | 0/20 [00:00<?, ?it/s]

#### load data

In [16]:
df = pd.read_parquet('./data/chosun_1980.parquet')

#### analysis

In [7]:
kkm = Kkma()

data = df.loc[:, 'body_archaic_hangul']

r = []
for text in tqdm(data):
    try:
        pos = kkm.pos(text)
    except Exception as error:
        print(error)
        print(text)
    for word, tag in pos:
        if tag.startswith('N'):
            r.append((word, tag))

  0%|          | 0/35907 [00:00<?, ?it/s]

'utf-8' codec can't decode byte 0xed in position 0: invalid continuation byte
본사내방(本社来訪)
▲이병희(李秉禧)씨(공화당부의장)▲조진희(趙晋熙)씨(조폐공사(造幣公社)사장)▲이태호(李泰浩)씨(한국수출입은행(韓國輸出入銀行)은행장)▲김건(金建)씨(동전무이사(同專務理事))▲김종섭(金鍾燮)씨(동감사(同監事))▲박승규(朴承圭)씨(환경청장)▲이우진(李愚振)씨(한국청년회의소중앙회장(韓國青年會議所中央會長))▲김정태(金政太)씨(동(同)상임부회장)▲이덕건(李德鍵)씨▲김국승(金國承)씨▲김응서(金應瑞)씨▲윤동렬(尹東烈)씨(동(同)부회장)▲백정환(白晶煥)씨(동사무총장(同事務總長))▲윤흥중(尹興重)씨(동(同)홍보실장)▲최규태(崔圭泰)씨(동(同)의전분과위원장)신임▲주철𤀹(朱鐵𤀹)씨(피어리스상무이사(常務理事))▲이병화(李炳華)씨(태평양화학홍보이사(太平洋化學弘報理事))▲김석년(金石年)씨(주(株))오리콤전무(專務))

'utf-8' codec can't decode byte 0xed in position 0: invalid continuation byte
조선일보(朝鮮日報) 신춘문예(新春文藝) 당선소설집(小說集) 출간
「신인(新人)」들의 문제작(問題作)‥34편 정선(精選)
 조선일보사(朝鮮日報社)는 창간60주년기념 출판의 하나로 조선일보문학선서(朝鮮日報文學選書) 제1,2,3집「조선일보(朝鮮日報) 신춘문예당선소설집(新春文藝當選小說集)」전3권을 펴냅니다.한국 신문 최초로 신춘문예 현상공모를시작한 조선일보(朝鮮日報)가 1929년부터 79년까지 약 50년간의신춘문예당(新春文藝當)—입선(入選)소설 70여편중에서 34편을 정선해 모은 소설작품집입니다.여기 수록된 소설들은 그때그때 치열한 경쟁을 통해서 첫째를 차지한,해마다 새로왔던 참신한 신인(新人)들의 문제작(問題作)으로서 한국 현대 단편소설의 발전단계를 몸으로 실중해주는 작품들이기도 합니다.또한 각편마다 그 시대의 사회상(社會相),문학적 특성들이 고루 고루 나타나 있읍니다.30년대 일제치

KeyboardInterrupt: 